# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/fahadalqhtani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/fahadalqhtani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/fahadalqhtani/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
print(f"Table name => {engine.table_names()}")
df = pd.read_sql("select * from InsertTableName", engine)
X = df['message']
# y = df[df.columns[~df.columns.isin(df[X.columns])]]
y = df.drop(['message', 'genre', 'id', 'original'], axis=1)

<ipython-input-2-abbaf2c5ac03>:3: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  print(f"Table name => {engine.table_names()}")


Table name => ['InsertTableName']


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline =  Pipeline([("vect", CountVectorizer(tokenizer= tokenize)),
                     ("tfidf", TfidfTransformer()),
                     ("moc", MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42 )

In [6]:
%%time
pipeline.fit(X_train, y_train)

CPU times: user 11.2 s, sys: 2.5 s, total: 13.7 s
Wall time: 46.3 s


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fdfd2830ee0>)),
                ('tfidf', TfidfTransformer()),
                ('moc',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pre = pipeline.predict(X_test)

In [8]:
def display_results(y_test, y_pre, itr):
    from sklearn.metrics import classification_report, confusion_matrix
    for i in range(itr):
            print(f"""
            Report the f1 score, precision and recall for each output category of the dataset.\n

            Class Name : {y_test.columns[i]}
            {classification_report(y_test.iloc[:,i], y_pre[:,i])}

            """)

In [9]:
display_results(y_test, y_pre, 36)


            Report the f1 score, precision and recall for each output category of the dataset.


            Class Name : related
                          precision    recall  f1-score   support

           0       0.72      0.27      0.39      1873
           1       0.80      0.97      0.88      5934
           2       0.59      0.40      0.47        58

    accuracy                           0.80      7865
   macro avg       0.71      0.54      0.58      7865
weighted avg       0.78      0.80      0.76      7865


            

            Report the f1 score, precision and recall for each output category of the dataset.


            Class Name : request
                          precision    recall  f1-score   support

           0       0.89      0.99      0.94      6533
           1       0.89      0.41      0.56      1332

    accuracy                           0.89      7865
   macro avg       0.89      0.70      0.75      7865
weighted avg       0.89      0.89      0.87    

/Users/fahadalqhtani/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fahadalqhtani/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fahadalqhtani/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu


            Report the f1 score, precision and recall for each output category of the dataset.


            Class Name : hospitals
                          precision    recall  f1-score   support

           0       0.99      1.00      1.00      7787
           1       0.00      0.00      0.00        78

    accuracy                           0.99      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.98      0.99      0.99      7865


            

            Report the f1 score, precision and recall for each output category of the dataset.


            Class Name : shops
                          precision    recall  f1-score   support

           0       1.00      1.00      1.00      7837
           1       0.00      0.00      0.00        28

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.99      1.00      0.99      7865


            

            Report the f1 score

/Users/fahadalqhtani/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'tfidf__use_idf': (True, False)
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2, n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
%%time
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/Users/fahadalqhtani/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


CPU times: user 1min 51s, sys: 41.1 s, total: 2min 32s
Wall time: 1h 52min 26s


In [12]:
display_results(y_test, y_pred, 36)


            Report the f1 score, precision and recall for each output category of the dataset.


            Class Name : related
                          precision    recall  f1-score   support

           0       0.70      0.27      0.39      1873
           1       0.80      0.96      0.88      5934
           2       0.53      0.34      0.42        58

    accuracy                           0.79      7865
   macro avg       0.68      0.53      0.56      7865
weighted avg       0.78      0.79      0.76      7865


            

            Report the f1 score, precision and recall for each output category of the dataset.


            Class Name : request
                          precision    recall  f1-score   support

           0       0.90      0.99      0.94      6533
           1       0.87      0.44      0.59      1332

    accuracy                           0.89      7865
   macro avg       0.88      0.71      0.76      7865
weighted avg       0.89      0.89      0.88    

/Users/fahadalqhtani/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fahadalqhtani/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fahadalqhtani/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu


            Report the f1 score, precision and recall for each output category of the dataset.


            Class Name : offer
                          precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00      0.00        36

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.99      1.00      0.99      7865


            

            Report the f1 score, precision and recall for each output category of the dataset.


            Class Name : aid_related
                          precision    recall  f1-score   support

           0       0.75      0.89      0.82      4646
           1       0.78      0.58      0.67      3219

    accuracy                           0.76      7865
   macro avg       0.77      0.73      0.74      7865
weighted avg       0.77      0.76      0.75      7865


            

            Report the f1 sco

/Users/fahadalqhtani/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fahadalqhtani/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fahadalqhtani/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu


            Report the f1 score, precision and recall for each output category of the dataset.


            Class Name : shops
                          precision    recall  f1-score   support

           0       1.00      1.00      1.00      7837
           1       0.00      0.00      0.00        28

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.99      1.00      0.99      7865


            

            Report the f1 score, precision and recall for each output category of the dataset.


            Class Name : aid_centers
                          precision    recall  f1-score   support

           0       0.99      1.00      0.99      7762
           1       0.00      0.00      0.00       103

    accuracy                           0.99      7865
   macro avg       0.49      0.50      0.50      7865
weighted avg       0.97      0.99      0.98      7865


            

            Report the f1 sco

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
from sklearn.neighbors import KNeighborsClassifier

pipeline =  Pipeline([("vect", CountVectorizer(tokenizer= tokenize)),
                     ("tfidf", TfidfTransformer()),
                     ("moc", MultiOutputClassifier((KNeighborsClassifier(n_jobs=-1))))])

In [14]:
%%time
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

CPU times: user 13min, sys: 8min 1s, total: 21min 2s
Wall time: 27min 12s


In [15]:
display_results(y_test, y_pred, 36)


            Report the f1 score, precision and recall for each output category of the dataset.


            Class Name : related
                          precision    recall  f1-score   support

           0       0.57      0.29      0.38      1873
           1       0.80      0.93      0.86      5934
           2       0.39      0.40      0.39        58

    accuracy                           0.77      7865
   macro avg       0.59      0.54      0.54      7865
weighted avg       0.74      0.77      0.74      7865


            

            Report the f1 score, precision and recall for each output category of the dataset.


            Class Name : request
                          precision    recall  f1-score   support

           0       0.84      1.00      0.91      6533
           1       0.81      0.06      0.11      1332

    accuracy                           0.84      7865
   macro avg       0.82      0.53      0.51      7865
weighted avg       0.83      0.84      0.78    

/Users/fahadalqhtani/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fahadalqhtani/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fahadalqhtani/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu


            Report the f1 score, precision and recall for each output category of the dataset.


            Class Name : tools
                          precision    recall  f1-score   support

           0       0.99      1.00      1.00      7817
           1       0.00      0.00      0.00        48

    accuracy                           0.99      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.99      0.99      0.99      7865


            

            Report the f1 score, precision and recall for each output category of the dataset.


            Class Name : hospitals
                          precision    recall  f1-score   support

           0       0.99      1.00      1.00      7787
           1       0.00      0.00      0.00        78

    accuracy                           0.99      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.98      0.99      0.99      7865


            

            Report the f1 score

/Users/fahadalqhtani/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fahadalqhtani/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fahadalqhtani/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

### 9. Export your model as a pickle file

In [16]:
import pickle
filename = 'My_model.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.